In [1]:
import pandas as pd
import sqlite3
import numpy as np

# Create a SQL connection to our SQLite database
con = sqlite3.connect("../DataOtherSources/EthereumNFTs/nfts.sqlite")

cur = con.cursor()
nftAddressNames = pd.read_sql_query('SELECT * from nfts', con)
# Be sure to close the connection
con.close()

In [2]:
nftAddressNames.loc[nftAddressNames["symbol"] == '','symbol'] = nftAddressNames["name"]

In [3]:
nftAddressNames["address"] = nftAddressNames["address"].str.lower()

In [5]:
nftAddressNames[nftAddressNames["address"]=="0xc8bcbe0e8ae36d8f9238cd320ef6de88784b1734"]

,address,name,symbol
6268,0xc8bcbe0e8ae36d8f9238cd320ef6de88784b1734,WinterBears,WB


In [6]:
import random
import string

def random_string(N=20):
    return ''.join(random.SystemRandom().choice(string.ascii_uppercase) for _ in range(N))+'VULNAAM'

nftAddressNames.update(nftAddressNames.loc[nftAddressNames["symbol"] == '','symbol'].apply(lambda x: random_string()))
nftAddressNames.update(nftAddressNames.loc[nftAddressNames["symbol"] != nftAddressNames["symbol"],'symbol'].apply(lambda x: random_string()))

In [7]:
nftAddressNames[nftAddressNames["address"]=="0xc8bcbe0e8ae36d8f9238cd320ef6de88784b1734"]

,address,name,symbol
6268,0xc8bcbe0e8ae36d8f9238cd320ef6de88784b1734,WinterBears,WB


In [8]:
nftAdress_grouped = nftAddressNames.groupby(["symbol"])

In [9]:
import numpy as np

nft_dict = {}

for (name, groupobject) in nftAdress_grouped.__iter__():
    topNft_Address = np.array(groupobject["address"])[0]
    Nft_addresses = np.array(groupobject["address"])
    for NFT_address in Nft_addresses:
        nft_dict[NFT_address] = topNft_Address

In [10]:
nftAddressNames["addressNameBaselined"] = nftAddressNames["address"].replace(nft_dict)

In [11]:
nftAddressNames["symbol"] = nftAddressNames["symbol"].apply( lambda x : "LEEG" if str(x).endswith('VULNAAM') else x )

In [13]:
nftAddressNames[nftAddressNames["address"]=="0xc8bcbe0e8ae36d8f9238cd320ef6de88784b1734"]

,address,name,symbol,addressNameBaselined
6268,0xc8bcbe0e8ae36d8f9238cd320ef6de88784b1734,WinterBears,WB,0xb653b39283084969717f53444d629c8a2d166cb1


In [14]:
from selenium import webdriver

def webScraperTokenName(ethTransaction_hash="0xFBeef911Dc5821886e1dda71586d90eD28174B7d"):
    print(ethTransaction_hash)
    url = 'https://etherscan.io/token/'+ethTransaction_hash
    browser = webdriver.Chrome()
    browser.get(url)
    nftSymbol = "-"
    nftName = "-"
    try:
        symb = browser.find_element_by_xpath('/html/head/meta[15]').get_property('content')
        if len(symb)>2: 
            symb = symb[symb.find("(")+1:symb.find(")")]
            nftSymbol = symb
    except Exception:
        print("err")
    try:
        name = browser.find_element_by_xpath('/html/body/div[1]/main/div[1]/div/div[1]/h1/div/span').text
        if name: nftName = name
    except Exception:
        print("err")

    browser.quit()
    return [nftSymbol,nftName]

In [15]:
npArray = nftAddressNames.to_numpy()

In [17]:
def myfunction(x):
    if x[2] != 'LEEG': return
    symb, name = webScraperTokenName(x[3])
    x[1] = name
    x[2] = symb

np.apply_along_axis(myfunction, axis=1, arr=npArray)

0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85


<ipython-input-14-3d9d4f9d750c>:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  symb = browser.find_element_by_xpath('/html/head/meta[15]').get_property('content')
<ipython-input-14-3d9d4f9d750c>:18: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  name = browser.find_element_by_xpath('/html/body/div[1]/main/div[1]/div/div[1]/h1/div/span').text


err
err


KeyboardInterrupt: 

In [24]:
nftAddressNames = pd.DataFrame(npArray)

In [25]:
nftAddressNames.columns = ["address", 'name', 'symbol', 'addressBaselined']

In [26]:
nftAddressNames

,address,name,symbol,addressBaselined
0,0xf4ee95274741437636e748ddac70818b4ed7d043,The Doge Pound,DOGGY,0xf4ee95274741437636e748ddac70818b4ed7d043
1,0xba30e5f9bb24caa003e9f2f0497ad287fdf95623,BoredApeKennelClub,BAKC,0xba30e5f9bb24caa003e9f2f0497ad287fdf95623
2,0x7403ac30de7309a0bf019cda8eec034a5507cbb3,Characters,CHAR,0x7403ac30de7309a0bf019cda8eec034a5507cbb3
3,0x511372b44231a31527025a3d273c1dc0a83d77af,Maps Restored,MAP,0x511372b44231a31527025a3d273c1dc0a83d77af
4,0x2e956ed3d7337f4ed4316a6e8f2edf74bf84bb54,Neon Junkies,JUNKIES,0x2e956ed3d7337f4ed4316a6e8f2edf74bf84bb54
...,...,...,...,...
9383,0x848433e170d7b253e8004415643260f18e36fa40,Fredrick and Penny,FNPBJ,0x848433e170d7b253e8004415643260f18e36fa40
9384,0xb5dc78d106cb4540ca945dc1fabc63845198f3f0,SH Minter,SH,0x33518f907bb9c36cd962438375a01dfe04edc1d4
9385,0x053bf84359415542e9d65138d1bc0dd0d01e10f7,NFT - 3,NFT3,0x053bf84359415542e9d65138d1bc0dd0d01e10f7
9386,0xc1a8621efdef8e2173435af416ac18d858ecaf49,Pedro Ferreira,PF,0xc1a8621efdef8e2173435af416ac18d858ecaf49


In [27]:
nftAddressNames[nftAddressNames["address"]=="0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85"]

,address,name,symbol,addressBaselined
50,0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85,None,LEEG,0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85


In [20]:
# nftAddressNames.to_csv("nftNames-baseline.csv",index=False)